In [ ]:
#5 
# update currency with drop_down
DEFAULTS = {'publication': 'bbc',
           'city': 'London, UK',
           'currency_from':'GBP',
           'currency_to':'USD'}

app = Flask(__name__)

RSS_FEEDS = {'bbc': 'http://feeds.bbci.co.uk/news/rss.xml',
             'cnn': 'http://rss.cnn.com/rss/edition.rss',
             'fox': 'http://feeds.foxnews.com/foxnews/latest',
             'iol': 'http://www.iol.co.za/cmlink/1.640'}

@app.route("/")
def home():
    
    #get customized currency
    currency_from = request.args.get("currency_from")
    if not currency_from:
        currency_from = DEFAULTS['currency_from']
        
    currency_to = request.args.get("currency_to")
    if not currency_to:
        currency_to = DEFAULTS['currency_to']
    rate, currencies = get_rate(currency_from, currency_to)
    
    return render_template("home_mod1.html",
                           articles=articles,
                          weather=weather,
                          currency_from=currency_from,
                          currency_to=currency_to,
                          rate=rate,
                          currencies=sorted(currencies))    
    
# 변형
def get_rate(frm, to):
    response = requests.get(currency_url)
    data = response.content
    parsed = json.loads(data).get('rates')
    frm_rate = parsed.get(frm.upper())
    to_rate = parsed.get(to.upper())
    return (to_rate/frm_rate, parsed.keys())

if __name__ == '__main__':
    app.run(port=5000, debug=True)

In [ ]:
#5 
# update currency info with select
import json
import urllib
import urllib3
import requests
import feedparser
import pickle
from flask import Flask, render_template, request

DEFAULTS = {'publication': 'bbc',
           'city': 'London, UK',
           'currency_from':'GBP',
           'currency_to':'USD'}

app = Flask(__name__)

RSS_FEEDS = {'bbc': 'http://feeds.bbci.co.uk/news/rss.xml',
             'cnn': 'http://rss.cnn.com/rss/edition.rss',
             'fox': 'http://feeds.foxnews.com/foxnews/latest',
             'iol': 'http://www.iol.co.za/cmlink/1.640'}

with open('weather_appID', 'rb') as r:
    we_appID = pickle.load(r)
weather_url = "http://api.openweathermap.org/data/2.5/weather?q={}&units=metric&APPID={}"

with open('currency_appID', 'rb') as r:
    cu_appID = pickle.load(r)
currency_url = "https://openexchangerates.org/api/latest.json/?app_id={}".format(cu_appID)

@app.route("/")
def home():
    # get customized headlines
    publication = request.args.get('publication')
    if not publication:
        publication = DEFAULTS['publication']
    articles = get_news(publication)
    
    # get customized weather
    city = request.args.get('city')
    if not city:
        city = DEFAULTS['city']
    weather = get_weather(city)
    
    #get customized currency
    currency_from = request.args.get("currency_from")
    if not currency_from:
        currency_from = DEFAULTS['currency_from']
        
    currency_to = request.args.get("currency_to")
    if not currency_to:
        currency_to = DEFAULTS['currency_to']
    rate = get_rate(currency_from, currency_to)
    return render_template("index.html",
                           articles=articles,
                          weather=weather,
                          currency_from=currency_from,
                          currency_to=currency_to,
                          rate=rate)    
    
# publication을 query로 받음
def get_news(query):
    if not query or query.lower() not in RSS_FEEDS:
        publication = DEFAULTS['publication']
    else:
        publication = query.lower()
    feed = feedparser.parse(RSS_FEEDS[publication])
    return feed['entries']

# cit를 query로 받음(그러나 query 문자를 변환하는 과정 필요함)
def get_weather(query):
    query = urllib.parse.quote(query)
    url = weather_url.format(query, we_appID)
    response = requests.get(url)
    data = response.content
    parsed = json.loads(data)
    weather = None
    if parsed.get("weather"):
        weather = {"DESC": parsed["weather"][0]["description"],
                  "temp": parsed["main"]["temp"],
                  "city": parsed["name"], 
                   "country": parsed['sys']['country']}
    return weather

def get_rate(frm, to):
    response = requests.get(currency_url)
    data = response.content
    parsed = json.loads(data).get('rates')
    frm_rate = parsed.get(frm.upper())
    to_rate = parsed.get(to.upper())
    return to_rate/frm_rate

if __name__ == '__main__':
    app.run(port=5000, debug=True)

# currency

In [26]:
import pickle

# with open('currency_appID', 'wb') as f:
#     pickle.dump(appID, f)
with open('currency_appID', 'rb') as r:
    cu_appID = pickle.load(r)

In [27]:
import requests
import json
currency_url = "https://openexchangerates.org/api/latest.json/?app_id={}".format(cu_appID)
response = requests.get(currency_url)
data = response.content
parsed = json.loads(data).get('rates')

In [53]:
parsed.keys()

dict_keys(['AED', 'AFN', 'ALL', 'AMD', 'ANG', 'AOA', 'ARS', 'AUD', 'AWG', 'AZN', 'BAM', 'BBD', 'BDT', 'BGN', 'BHD', 'BIF', 'BMD', 'BND', 'BOB', 'BRL', 'BSD', 'BTC', 'BTN', 'BWP', 'BYN', 'BZD', 'CAD', 'CDF', 'CHF', 'CLF', 'CLP', 'CNH', 'CNY', 'COP', 'CRC', 'CUC', 'CUP', 'CVE', 'CZK', 'DJF', 'DKK', 'DOP', 'DZD', 'EGP', 'ERN', 'ETB', 'EUR', 'FJD', 'FKP', 'GBP', 'GEL', 'GGP', 'GHS', 'GIP', 'GMD', 'GNF', 'GTQ', 'GYD', 'HKD', 'HNL', 'HRK', 'HTG', 'HUF', 'IDR', 'ILS', 'IMP', 'INR', 'IQD', 'IRR', 'ISK', 'JEP', 'JMD', 'JOD', 'JPY', 'KES', 'KGS', 'KHR', 'KMF', 'KPW', 'KRW', 'KWD', 'KYD', 'KZT', 'LAK', 'LBP', 'LKR', 'LRD', 'LSL', 'LYD', 'MAD', 'MDL', 'MGA', 'MKD', 'MMK', 'MNT', 'MOP', 'MRO', 'MRU', 'MUR', 'MVR', 'MWK', 'MXN', 'MYR', 'MZN', 'NAD', 'NGN', 'NIO', 'NOK', 'NPR', 'NZD', 'OMR', 'PAB', 'PEN', 'PGK', 'PHP', 'PKR', 'PLN', 'PYG', 'QAR', 'RON', 'RSD', 'RUB', 'RWF', 'SAR', 'SBD', 'SCR', 'SDG', 'SEK', 'SGD', 'SHP', 'SLL', 'SOS', 'SRD', 'SSP', 'STD', 'STN', 'SVC', 'SYP', 'SZL', 'THB', 'TJS', 'T

In [50]:
def get_rate(frm, to):
    response = requests.get(currency_url)
    data = response.content
    parsed = json.loads(data).get('rates')
    frm_rate = parsed.get(frm.upper())
    to_rate = parsed.get(to.upper())
    return to_rate/frm_rate

In [55]:
get_rate("USD", "KRW")

1127.508333

---

In [ ]:
#4 makedefault & separate func
import json
import urllib
import urllib3
import requests
import feedparser
from flask import Flask, render_template, request

DEFAULTS = {'publication': 'bbc',
           'city': 'London, UK'}
app = Flask(__name__)

RSS_FEEDS = {'bbc': 'http://feeds.bbci.co.uk/news/rss.xml',
             'cnn': 'http://rss.cnn.com/rss/edition.rss',
             'fox': 'http://feeds.foxnews.com/foxnews/latest',
             'iol': 'http://www.iol.co.za/cmlink/1.640'}

with open('weather_appID', 'rb') as r:
    we_appID = pickle.load(r)
    
weather_url = "http://api.openweathermap.org/data/2.5/weather?q={}&units=metric&APPID={}"
@app.route("/")
def home():
    # get customized headlines
    publication = request.args.get('publication')
    if not publication:
        publication = DEFAULTS['publication']
    articles = get_news(publication)
    
    # get customized weather
    city = request.args.get('city')
    if not city:
        city = DEFAULTS['city']
    weather = get_weather(city)
    return render_template("index.html",
                           articles=articles,
                          weather=weather)    
    
# publication을 query로 받음
def get_news(query):
    if not query or query.lower() not in RSS_FEEDS:
        publication = DEFAULTS['publication']
    else:
        publication = query.lower()
    feed = feedparser.parse(RSS_FEEDS[publication])
    return feed['entries']

# cit를 query로 받음(그러나 query 문자를 변환하는 과정 필요함)
def get_weather(query):
    query = urllib.parse.quote(query)
    url = weather_url.format(query, we_appID)
    response = requests.get(url)
    data = response.content
    parsed = json.loads(data)
    weather = None
    if parsed.get("weather"):
        weather = {"DESC": parsed["weather"][0]["description"],
                  "temp": parsed["main"]["temp"],
                  "city": parsed["name"], 
                   "country": parsed['sys']['country']}
    return weather

if __name__ == '__main__':
    app.run(port=5000, debug=True)

In [ ]:
#3
## update weather function
import feedparser
from flask import Flask, render_template, request

app = Flask(__name__)

RSS_FEEDS = {'bbc': 'http://feeds.bbci.co.uk/news/rss.xml',
             'cnn': 'http://rss.cnn.com/rss/edition.rss',
             'fox': 'http://feeds.foxnews.com/foxnews/latest',
             'iol': 'http://www.iol.co.za/cmlink/1.640'}

@app.route("/", methods=["GET", "POST"])
def get_news():
    query = request.form.get("publication")
    if not query or query.lower() not in RSS_FEEDS:
        publication = "bbc"
    else:
        publication = query.lower()
        
    feed = feedparser.parse(RSS_FEEDS[publication])
    
    weather = get_weather("London, UK")
    return render_template("index.html",
                           articles=feed['entries'],
                          weather=weather)

if __name__ == '__main__':
    app.run(port=5000, debug=True)

---

# weather

In [25]:
import json
import urllib
import requests
with open('weather_appID', 'rb') as r:
    we_appID = pickle.load(r)
weather_url = "http://api.openweathermap.org/data/2.5/weather?q={}&units=metric&APPID={}".format(we_appID)

def get_weather(query):
    query = urllib.parse.quote(query)
    url = api_url.format(query)
    response = requests.get(url)
    data = response.content
    parsed = json.loads(data)
    weather = None
    
    if parsed.get("weather"):
        weather = {"DESC": parsed["weather"][0]["description"],
                  "temp": parsed["main"]["temp"],
                  "city": parsed["name"]}
    return weather
    

In [26]:
get_weather(q)

{'DESC': 'clear sky', 'temp': 11.8, 'city': 'London'}

In [25]:
# with open('weather_appID', 'wb') as f:
#     pickle.dump(appID, f)
with open('weather_appID', 'rb') as r:
    we_appID = pickle.load(r)
# weather_url = "http://api.openweathermap.org/data/2.5/weather?q={}&units=metric&APPID={}".format(we_appID)


In [24]:
q = 'London, UK'

api_url = "http://api.openweathermap.org/data/2.5/weather?q={}&units=metric&APPID={}".format(we_appID)
query = urllib.parse.quote(q)
url = weather_url.format(query)
response = requests.get(url)
data = response.content
parsed = json.loads(data)
parsed

{'coord': {'lon': -0.13, 'lat': 51.51},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01d'}],
 'base': 'stations',
 'main': {'temp': 11.8,
  'pressure': 1015,
  'humidity': 58,
  'temp_min': 11,
  'temp_max': 13},
 'visibility': 10000,
 'wind': {'speed': 5.1, 'deg': 200},
 'clouds': {'all': 0},
 'dt': 1550402400,
 'sys': {'type': 1,
  'id': 1414,
  'message': 0.0053,
  'country': 'GB',
  'sunrise': 1550387416,
  'sunset': 1550423972},
 'id': 2643743,
 'name': 'London',
 'cod': 200}

In [23]:
# response.content: Non-json
json.loads(response.content)

{'coord': {'lon': -0.13, 'lat': 51.51},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01d'}],
 'base': 'stations',
 'main': {'temp': 11.8,
  'pressure': 1015,
  'humidity': 58,
  'temp_min': 11,
  'temp_max': 13},
 'visibility': 10000,
 'wind': {'speed': 5.1, 'deg': 200},
 'clouds': {'all': 0},
 'dt': 1550402400,
 'sys': {'type': 1,
  'id': 1414,
  'message': 0.0053,
  'country': 'GB',
  'sunrise': 1550387416,
  'sunset': 1550423972},
 'id': 2643743,
 'name': 'London',
 'cod': 200}

---

In [ ]:
#2 post
## .html: <form action="/" method="POST">
## .py: request.form.get
import feedparser
from flask import Flask, render_template, request

app = Flask(__name__)

RSS_FEEDS = {'bbc': 'http://feeds.bbci.co.uk/news/rss.xml',
             'cnn': 'http://rss.cnn.com/rss/edition.rss',
             'fox': 'http://feeds.foxnews.com/foxnews/latest',
             'iol': 'http://www.iol.co.za/cmlink/1.640'}

@app.route("/", methods=["GET", "POST"])
def get_news():
    query = request.form.get("publication")
    if not query or query.lower() not in RSS_FEEDS:
        publication = "bbc"
    else:
        publication = query.lower()
        
    feed = feedparser.parse(RSS_FEEDS[publication])
    return render_template("index.html",
                           articles=feed['entries'])

if __name__ == '__main__':
    app.run(port=5000, debug=True)

In [ ]:
#1
## .html:<form> element에서 get 방식으로 요청하는 것을 활용 eg. /?query(key)=value (key=query:"publication")
## .py: falsk_request는 가장 최신의 reqeust_dict(key & value)를 args로 저장함(request.args.get(key))
import feedparser
from flask import Flask, render_template, request

app = Flask(__name__)

RSS_FEEDS = {'bbc': 'http://feeds.bbci.co.uk/news/rss.xml',
             'cnn': 'http://rss.cnn.com/rss/edition.rss',
             'fox': 'http://feeds.foxnews.com/foxnews/latest',
             'iol': 'http://www.iol.co.za/cmlink/1.640'}

@app.route("/")
def get_news():
    query = request.args.get("publication")
    if not query or query.lower() not in RSS_FEEDS:
        publication = "bbc"
    else:
        publication = query.lower()
        
    feed = feedparser.parse(RSS_FEEDS[publication])
    return render_template("index.html",
                           articles=feed['entries'])

if __name__ == '__main__':
    app.run(port=5000, debug=True)

In [ ]:
import feedparser
bbc_url = "http://feeds.bbci.co.uk/news/rss.xml"
feed = feedparser.parse(bbc_url)
feed.keys()